# LSTM实践

循环神经网络的变种LSTM模型也有很多种结构，从常规的lstm，到各类复杂结构，这里根据实际使用情况，以pytorch为主，也会有tf2的例子，逐步记录。

## vanilla lstm

常规的lstm模型。

lstm模型的相关基础知识可以参考：[Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)。

这里简单总结下。首先RNN的结构如图所示：

![](RNN-unrolled.png)

RNN中A就是一个激活函数tanh。两个A之间的箭头意思是把第一个A输出的h传递给第二个A。也就是说，第二个A的输入是concatenate了上一个时段的输出和本时段的输入。

那么LSTM就是将A进一步优化，引入了Cell State，通过几个gate来控制节点 来决定记忆哪些，忘记哪些，其中一个四门的形式的RNN就是LSTM。四门如下：

![](Picture10.png)

橙色是forget gate layer。

黄色包括两小部分，是为了确定要记忆什么，第一小部分是input gate layer；第二小部分是初始记忆单元作为初始cell state。

绿色是更新cell state，结合遗忘后剩下的历史cell state和刚黄色部分决定本次记忆的cell state 组成新的cell state

蓝色就是最后利用新生成的cell state，结合上个时段的h，生成新的h。

所以和RNN不同，LSTM不同时间段之间的A之间的联系包括h和cell state两种。

还有一个内容比较重要，需要了解下，就是dropout。

参考：[Dropout在RNN中的应用综述](https://lonepatient.top/2018/09/24/a-review-of-dropout-as-applied-to-rnns.html)，及原文[A review of Dropout as applied to RNNs](https://medium.com/@bingobee01/a-review-of-dropout-as-applied-to-rnns-72e79ecd5b7b)

前面已经说过了Dropout基本概念。

RNN与仅前馈神经网络的不同之处在于**先前的状态反馈到网络中，允许网络保留先前状态**。因此，将标准dropout应用于RNN会**限制网络保留先前状态的能力**，从而阻碍其性能。 Bayer等人指出了将dropout应用于递归神经网络（RNN）的问题。如果将完整的输出权重向量设置为零，则“在每次前向传递期间导致RNN动态变化是非常显著的。”

首先看下RNN的前向计算代码：

```python
class RNN:
# ...
    def step(self, x):
        # update the hidden state
        self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
        # compute the output vector
        y = np.dot(self.W_hy, self.h)
        return y
rnn = RNN()
y = rnn.step(x) # x is an input ve
```

作为克服应用于RNN的dropout性能问题的一种方法，Zaremba和Pham等仅将dropout应用于非循环连接（Dropout未应用于隐藏状态）。 “通过不对循环连接使用dropout，LSTM可以从dropout正则化中受益，而不会牺牲其宝贵的记忆能力。”

![](79805875.jpg)

即 Dropout仅适用于非循环连接（即仅应用于前馈虚线）。 粗线显示了LSTM中典型的信息流路径。 信息受到dropoutL + 1次的影响，其中L是网络的深度。

Gal和Ghahramani（2015）分析了将Dropout应用于仅前馈RNN的部分，发现这种方法仍然导致过拟合。 他们提出了“**变分Dropout**”，通过**重复“输入，输出和循环层的每个时间步长相同的dropout掩码（在每个时间步骤丢弃相同的网络单元）**”，使用贝叶斯解释，他们看到了语言的改进 建模和情感分析任务超过’纯dropout’。

![](14161715.jpg)

即朴素dropout（a）在不同的时间步长使用不同的掩模，在循环层上没有dropout。 变分dropout（b）在每个时间步长使用相同的压差掩模，包括循环层（颜色表示Dropout掩模，实线表示dropout，虚线表示没有dropout的标准连接）。

与Moon和Gal和Ghahramani一样，Semeniuta等人提出**将dropout应用于RNN的循环连接**，以便可以**对循环权重进行正则化以提高性能**。 Gal和Ghahramani使用**网络的隐藏状态**作为计算门值和小区更新以及使用dropout的子网络的输入来正则化子网络（上图b）。 Semeniuta等人的不同之处在于他们认为“整个架构以隐藏状态为关键部分并使整个网络正则化”。 这类似于Moon等人的概念（如上图a所示），但Semeniuta等人发现根据Moon等人直接丢弃先前的状态产生了混合结果，而将dropout应用于隐藏状态更新向量是一种更有原则的方法。

![](52714967.jpg)

上图是 Semeniuta等人论文中在LSTM网络的循环连接中三种类型的dropout的例证。 虚线箭头表示断开的连接。 为清楚起见，省略了输入连接。“ 注意Semeniuta等人如何。 （2016）将重复dropout应用于LSTM存储器单元的更新。

设置权重dropout的代码关键部分：

In [ ]:
def _setweights(self):
    for name_w in self.weights:
        raw_w = getattr(self.module, name_w + '_raw')
        w = None
        if self.variational:
            mask = torch.autograd.Variable(torch.ones(raw_w.size(0), 1))
            if raw_w.is_cuda: mask = mask.cuda()
            mask = torch.nn.functional.dropout(mask, p=self.dropout,
             training=True)
            w = mask.expand_as(raw_w) * raw_w
        else:
            w = torch.nn.functional.dropout(raw_w, p=self.dropout,
         training=self.training)
        setattr(self.module, name_w, w)

可以参考一个实例进一步了解，这里就不赘述了--博客 [How to Develop Multi-Step LSTM Time Series Forecasting Models for Power Usage](https://machinelearningmastery.com/how-to-develop-lstm-models-for-multi-step-time-series-forecasting-of-household-power-consumption/)

## stacked lstm

先对stacked lstm有个基本认识，参考：[Single Layer & Multi-layer Long Short-Term Memory (LSTM) Model with Intermediate Variables for Weather Forecasting](https://www.sciencedirect.com/science/article/pii/S187705091831439X)，一个典型的stacked lstm结构是这样的：

![](Picture9.png)

接下来参考[Stacked Long Short-Term Memory Networks](https://machinelearningmastery.com/stacked-long-short-term-memory-networks/)记录一些基本内容。

原始的lstm就是由一个LSTM隐含层和一个标准前向输出层组成。 Stacked LSTM 是对该模型的扩充，它有多个LSTM隐含层，每层有多个记忆单元。

### 为什么要增加深度

神经网络的深度通常被认为是该方法在一系列具有挑战性的预测问题上取得成功的原因。更多的隐含层能让神经网络更深。更深的隐含层目前被认为能从之前的层中重新组合学习到的特征并创建新的更高层次的抽象特征。比如从线到面到体。

一个足够深的神经网络结构能用来估计大部分函数。增加网络的深度提供了一种需要更少的神经元和更快的训练的解决方案。因此，增加深度是一种典型的优化方式。

### Stacked LSTM Architecture

因为LSTM是作用于序列数据，这意味着随着时间推移，层数的增加增加了输入观察的抽象级别。实际上就是将观察结果随时间分块或在不同的时间尺度上表示问题。

引用一段话：

" ... building a deep RNN by stacking multiple recurrent hidden states on top of each other. This approach potentially allows the hidden state at each level to operate at different timescale"

— [How to Construct Deep Recurrent Neural Networks, 2013](https://arxiv.org/abs/1312.6026)

在一些研究中，LSTM的深度比一个给定层的memory cells的个数对模型来说更重要。Stacked LSTMs 现在已经是序列预测问题的一种较稳定的技术。一个Stacked LSTM模型可以由一个LSTM模型和多个LSTM层构成。如下图所示，上面的LSTM层不输出一个值，而是输出一个序列到下一层LSTM。具体来说，每个输入时间步一个输出，而不是所有输入时间步一个输出。

### Implement Stacked LSTMs

在Keras中可以很容易的创建Stacked LSTM，每个LSTM memory cell 需要一个3D输入，当LSTM处理一个时间步长的输入序列时，每个存储单元将以2D数组的形式输出整个序列的单个值。这部分可以参考上面提到的博客，这里不再赘述。

提示一点：可以看到为了堆叠LSTM层，需要修改LSTM层输出一个3D的数组作为下一层的输入。设置return_sequences=True，它就会输出一个序列。

然后就可以增加LSTM层了。比如：

```python
model = Sequential()
model.add(LSTM(..., return_sequences=True, input_shape=(...)))
model.add(LSTM(...))
model.add(Dense(...))
```

或者更多层：

```python
model = Sequential()
model.add(LSTM(..., return_sequences=True, input_shape=(...)))
model.add(LSTM(..., return_sequences=True))
model.add(LSTM(..., return_sequences=True))
model.add(LSTM(...))
model.add(Dense(...))
```